https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

[Data dictionary](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

In [3]:
import pandas as pd
import pyarrow
# import fastparquet

url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-12.parquet"
df2 = pd.read_parquet(url)

In [30]:
month_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
year_list = [str(i) for i in range(2009, 2023)]

for yeear in year_list:
    for month in month_list

['2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

In [4]:
df2.shape
df2

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-12-01 00:37:35,2022-12-01 00:47:35,1.0,2.00,1.0,N,170,237,1,8.50,3.00,0.5,3.10,0.0,0.3,15.40,2.5,0.00
1,1,2022-12-01 00:34:35,2022-12-01 00:55:21,0.0,8.40,1.0,N,138,141,2,26.00,4.25,0.5,0.00,0.0,0.3,31.05,2.5,1.25
2,1,2022-12-01 00:33:26,2022-12-01 00:37:34,1.0,0.80,1.0,N,140,140,1,5.00,3.00,0.5,1.76,0.0,0.3,10.56,2.5,0.00
3,1,2022-12-01 00:45:51,2022-12-01 00:53:16,1.0,3.00,1.0,N,141,79,3,10.00,3.00,0.5,0.00,0.0,0.3,13.80,2.5,0.00
4,2,2022-12-01 00:49:49,2022-12-01 00:54:13,1.0,0.76,1.0,N,261,231,1,5.00,0.50,0.5,1.76,0.0,0.3,10.56,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399544,2,2022-12-31 23:46:00,2023-01-01 00:11:00,NaN,11.56,NaN,None,16,36,0,39.55,0.00,0.5,8.21,0.0,1.0,49.26,NaN,NaN
3399545,2,2022-12-31 23:13:24,2022-12-31 23:29:08,NaN,5.06,NaN,None,75,50,0,26.23,0.00,0.5,0.00,0.0,1.0,30.23,NaN,NaN
3399546,2,2022-12-31 23:00:49,2022-12-31 23:26:57,NaN,13.35,NaN,None,168,197,0,47.73,0.00,0.5,9.85,0.0,1.0,59.08,NaN,NaN
3399547,1,2022-12-31 23:02:50,2022-12-31 23:16:05,NaN,0.00,NaN,None,238,116,0,12.74,0.00,0.5,0.00,0.0,1.0,16.74,NaN,NaN


In [27]:
df = df2.head(100000).copy()

# Remove edge cases
df[df['tip_amount'] < 1000] # Remove data entry error with 9-figure tip

# Create a column for trip duration
duration_list = []
time_delta_series = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
for t in time_delta_series:
    duration_list.append(t.total_seconds() / 60)
df.insert(3, 'duration', duration_list)


# Define date from the pickup time
df.insert(0, 'date', df['tpep_pickup_datetime'].dt.date)

# Define type
df.insert(1, 'type', 'yellow_taxi')

# Indicate if the ride was to an airport (RatecodeID = 2 or 3 or airport_fee is non-zero)
df['airport_share'] = df.apply(lambda x: 1 if x['RatecodeID'] in [2, 3] or x['airport_fee'] > 0 else 0, axis=1)

# Indicate if the payment was credit
df['credit_share'] = df['payment_type'].apply(lambda x: 1 if x == 1 else 0)

# Define columns to keep
columns_to_keep =  ['date', 'type', 'duration', 'passenger_count', 'trip_distance', 'tip_amount', 'total_amount', 'airport_share', 'credit_share']
df = df[columns_to_keep]

df_daily = df.groupby(['date', 'type']).mean()
df_daily.insert(0, 'count', df.groupby(['date', 'type']).count()['duration'])

df_daily

,,count,duration,passenger_count,trip_distance,tip_amount,total_amount,airport_share,credit_share
date,type,,,,,,,,
2022-11-30,yellow_taxi,41,48.808943,1.829268,4.061951,3.546585,24.558780,0.097561,0.853659
2022-12-01,yellow_taxi,130584,19.615969,1.342595,3.762795,2.979600,22.982356,0.097968,0.774337
2022-12-02,yellow_taxi,129951,18.574828,1.409842,7.078983,2.839394,22.101694,0.090442,0.764111
2022-12-03,yellow_taxi,129172,17.765761,1.486262,7.828807,2.695179,20.934024,0.070743,0.764252
2022-12-04,yellow_taxi,111619,17.331685,1.459053,5.187190,3.014759,23.263871,0.119693,0.764637
2022-12-05,yellow_taxi,113626,17.839936,1.351442,6.654190,2.937978,22.739217,0.114965,0.762317
2022-12-06,yellow_taxi,120179,19.894612,1.321348,5.898933,3.026158,22.935720,0.093394,0.772614
2022-12-07,yellow_taxi,125589,20.066481,1.327339,5.750370,3.096526,23.262348,0.095422,0.782306
2022-12-08,yellow_taxi,133862,20.442521,1.338319,4.976634,3.129861,23.638735,0.095285,0.781088


In [14]:
df.columns

Index(['date', 'type', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'duration', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee', 'airport'],
      dtype='object')